# Notebook Memory Networks - Question Answering

### The dataset can be downloaded - https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz

In [7]:
# import the necessary packages

from keras.layers import Input
from keras.layers.core import Dense, Activation, Dropout, Permute
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.models import Model

import numpy as np
import itertools
import matplotlib.pyplot as plt

%matplotlib inline

### Read the files to create train and test data

In [17]:
TRAIN_FILE = "/Users/tkmacl9/Desktop/FastAIDLCourse/nbs/datasets/FB_BABL_Ques_Answering/tasks_1-20_v1-2/en/qa1_single-supporting-fact_train.txt"
TEST_FILE = "/Users/tkmacl9/Desktop/FastAIDLCourse/nbs/datasets/FB_BABL_Ques_Answering/tasks_1-20_v1-2/en/qa1_single-supporting-fact_test.txt"

stories = []
ques = []
answers = []
def parseFile(infile):
    story = []
    # This parses each line and creates the stories, questions and answers tuple
    for line in open(infile):
        content = line.split(" ", 1)
        if "\t" in line:
            stor_ques_ans = line.split("\t")
            ques.append(stor_ques_ans[1])
            answers.append(stor_ques_ans[2])
            stories.append(story)
            story = []
        else:
            story.append(content[1])
    return stories, ques, answers
    
train_data = parseFile(TRAIN_FILE)
test_data = parseFile(TEST_FILE)

In [19]:
print(train_data[0])
print(" ")
print(test_data[0])

[['Mary moved to the bathroom.\n', 'John went to the hallway.\n'], ['Daniel went back to the hallway.\n', 'Sandra moved to the garden.\n'], ['John moved to the office.\n', 'Sandra journeyed to the bathroom.\n'], ['Mary moved to the hallway.\n', 'Daniel travelled to the office.\n'], ['John went back to the garden.\n', 'John moved to the bedroom.\n'], ['Sandra travelled to the office.\n', 'Sandra went to the bathroom.\n'], ['Mary went to the bedroom.\n', 'Daniel moved to the hallway.\n'], ['John went to the garden.\n', 'John travelled to the office.\n'], ['Daniel journeyed to the bedroom.\n', 'Daniel travelled to the hallway.\n'], ['John went to the bedroom.\n', 'John travelled to the office.\n'], ['Mary went to the bedroom.\n', 'John journeyed to the bathroom.\n'], ['Sandra journeyed to the hallway.\n', 'John journeyed to the garden.\n'], ['John journeyed to the bathroom.\n', 'Sandra journeyed to the garden.\n'], ['Sandra went back to the bedroom.\n', 'Daniel travelled to the bathroom.\

### Build Vocab from traina and test data

### Get Maximum Lengths for the story and Question

### Vectorize the Inputs and Pad the sequences

### Define the Model Architecture